In [1]:
import numpy as np
from skimage import io, filters, img_as_float
from skimage.color import rgb2gray
from skimage import io, filters, img_as_float
from skimage.color import rgb2gray
from skimage import io, img_as_ubyte

# Convert composite image to 8-bit unsigned integer format



def ensure_gray(image):
    # Check if the image has three channels
    if image.ndim == 3 and image.shape[-1] == 3:
        return rgb2gray(image)
    else:
        # Assume image is already grayscale
        return image

# List of image file paths (ensure correct paths with raw strings or escaped backslashes)
image_paths = [
    r"C:\Projects\Embryo\Dataset\embryo_dataset_F15\AG274-2\D2013.03.13_S0763_I132_WELL2_RUN148.jpeg",
    r"C:\Projects\Embryo\Dataset\embryo_dataset_F-15\AG274-2\D2013.03.13_S0763_I132_WELL2_RUN148.jpeg",
    r"C:\Projects\Embryo\Dataset\embryo_dataset_F30\AG274-2\D2013.03.13_S0763_I132_WELL2_RUN148.jpeg",
    r"C:\Projects\Embryo\Dataset\embryo_dataset_F-30\AG274-2\D2013.03.13_S0763_I132_WELL2_RUN148.jpeg",
    r"C:\Projects\Embryo\Dataset\embryo_dataset_F45\AG274-2\D2013.03.13_S0763_I132_WELL2_RUN148.jpeg",
    r"C:\Projects\Embryo\Dataset\embryo_dataset_F-45\AG274-2\D2013.03.13_S0763_I132_WELL2_RUN148.jpeg"
]

# Load images and convert to float format
images = [img_as_float(io.imread(path)) for path in image_paths]

# Convert images to grayscale only if they are RGB
gray_images = [ensure_gray(image) for image in images]

# Define a function to compute the focus measure using the Laplacian operator
def compute_focus_measure(image):
    return np.abs(filters.laplace(image))

# Compute focus measures for all images
focus_measures = [compute_focus_measure(g) for g in gray_images]

# Stack focus measures into a single array of shape (height, width, num_images)
focus_stack = np.stack(focus_measures, axis=-1)

# Find the index of the image with the maximum focus measure for each pixel
max_focus_indices = np.argmax(focus_stack, axis=-1)

# Prepare an empty composite image (assumes images are RGB; adjust for grayscale if needed)
height, width = gray_images[0].shape
composite = np.zeros((height, width, 3))

# Build the composite image by selecting pixels from the image with the highest focus measure
# Loop over each pixel (for clarity; for large images consider vectorized approaches)
for i in range(height):
    for j in range(width):
        idx = max_focus_indices[i, j]
        composite[i, j] = images[idx][i, j]

composite_uint8 = img_as_ubyte(composite)

# Save the composite image
io.imsave("embryo_composite1.jpg", composite_uint8)


In [2]:
import cv2
import numpy as np

# List of image file paths (use raw strings to avoid issues with backslashes)
image_paths = [
    r"C:\Projects\Embryo\Dataset\embryo_dataset_F15\AG274-2\D2013.03.13_S0763_I132_WELL2_RUN148.jpeg",
    r"C:\Projects\Embryo\Dataset\embryo_dataset_F-15\AG274-2\D2013.03.13_S0763_I132_WELL2_RUN148.jpeg",
    r"C:\Projects\Embryo\Dataset\embryo_dataset_F30\AG274-2\D2013.03.13_S0763_I132_WELL2_RUN148.jpeg",
    r"C:\Projects\Embryo\Dataset\embryo_dataset_F-30\AG274-2\D2013.03.13_S0763_I132_WELL2_RUN148.jpeg",
    r"C:\Projects\Embryo\Dataset\embryo_dataset_F45\AG274-2\D2013.03.13_S0763_I132_WELL2_RUN148.jpeg",
    r"C:\Projects\Embryo\Dataset\embryo_dataset_F-45\AG274-2\D2013.03.13_S0763_I132_WELL2_RUN148.jpeg"
]

# Load images using OpenCV (images are loaded in BGR format)
images = [cv2.imread(path) for path in image_paths]

# Convert images to grayscale if they are not already grayscale
gray_images = []
for img in images:
    # Check if the image is already grayscale
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img
    gray_images.append(gray)

# Compute the focus measure using the absolute value of the Laplacian for each grayscale image
laplacian_images = [np.abs(cv2.Laplacian(gray, cv2.CV_64F)) for gray in gray_images]

# Stack the focus measures into a 3D array of shape (height, width, number_of_images)
laplacian_stack = np.stack(laplacian_images, axis=-1)

# For each pixel, find the index of the image with the highest focus measure
max_focus_indices = np.argmax(laplacian_stack, axis=-1)

# Create an empty composite image with the same size and number of channels as the original images
height, width = gray_images[0].shape
composite = np.zeros_like(images[0])

# For every pixel, select the pixel from the image with the highest focus measure
for i in range(height):
    for j in range(width):
        idx = max_focus_indices[i, j]
        composite[i, j] = images[idx][i, j]

# Save the composite image
cv2.imwrite("embryo_composite2.jpg", composite)


True